# Thresholding Downdraft Vertical Velocity

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import glob
import xoak
import geopandas as gpd
import regionmask

import warnings
warnings.filterwarnings('ignore')

In [2]:
#find directories with the WRF-BCC data
uh_dirts = glob.glob('/home/scratch/WRF_BCC/severe_weather/UP_HELI_MAX/historical/*')
uh_dirts.sort()
uh_dirts = uh_dirts[:-1]

ref_dirts = glob.glob('/home/scratch/WRF_BCC/reflectivity/REFD/historical/*')
ref_dirts.sort()
ref_dirts = ref_dirts[:]

dvv_dirts = glob.glob('/home/scratch/WRF_BCC/severe_weather/W_DN_MAX/historical/*')
dvv_dirts.sort()

uvv_dirts = glob.glob('/home/scratch/WRF_BCC/severe_weather/W_UP_MAX/historical/*')
uvv_dirts.sort()

### Mask US Data

In [3]:
#load the WRF-BCC geog and a random refc dummy file
geog = xr.open_dataset("/home/scratch/WRF_BCC/geography/geo_em.d01.nc")
ds = xr.open_mfdataset('/home/scratch/WRF_BCC/reflectivity/REFD/historical/1990-1991/*.nc')

#merge the files and create needed infomation
ds = xr.merge([ds, geog.squeeze()])
ds = ds.rename({"CLONG": 'lon', 'CLAT': 'lat'})
ds = ds.assign_coords({'x': ds.west_east, 'y': ds.south_north})
ds = ds.assign_coords({'lon': ds.lon, 'lat': ds.lat})

#set the lat-lon as the index
ds.xoak.set_index(['lat', 'lon'], 'sklearn_geo_balltree')

In [4]:
#load an USA shapefile
usa = gpd.read_file("/home/jcorner1/Unidata/shapefiles/smoothing_econus.shp")

#mask the data out
state_mask = regionmask.mask_geopandas(usa, ds.lon, ds.lat)
ma = state_mask.values
ma[~np.isnan(ma)] = 1

### Threshold Data

In [5]:
#iterate through each year (directory)
for dirt_number in range(len(uh_dirts)):
    for month in [10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    
        #open all the data within the directory
        print(f'current year and month: {int(uh_dirts[dirt_number][-4:])-1}, {month}')
        uh_ds = xr.open_mfdataset(glob.glob(f'{uh_dirts[dirt_number]}/*{uh_dirts[dirt_number][-4:]}_*-{str(month).zfill(2)}-*.nc'))
        ref_ds = xr.open_mfdataset(glob.glob(f'{ref_dirts[dirt_number]}/*{ref_dirts[dirt_number][-4:]}_*-{str(month).zfill(2)}-*.nc'))
        dvv_ds = xr.open_mfdataset(glob.glob(f'{dvv_dirts[dirt_number]}/*{dvv_dirts[dirt_number][-4:]}_*-{str(month).zfill(2)}-*.nc'))
        uvv_ds = xr.open_mfdataset(glob.glob(f'{uvv_dirts[dirt_number]}/*-{str(month).zfill(2)}-*.nc'))

        #grab all values
        uh_val = uh_ds.UP_HELI_MAX.values
        refc_val = ref_ds.REFD.values
        dvv_val = dvv_ds.W_DN_MAX.values
        uvv_val = uvv_ds.W_UP_MAX.values

        #subset times
        times = uh_ds.Time.values

        #threshold upward vertical velocities, reflectivity, and updraft helicity values. 
        thr_dvv = dvv_ds.where(dvv_val * -1 >= 5.0, 0)
        thr_dvv = thr_dvv.where(thr_dvv.W_DN_MAX.values * -1 <= 5.0, 1)

        #Give all areas with a UH of 75+ and reflectivity of 50+ a value of 1. 
        thr_val = thr_dvv.W_DN_MAX.values * ma

        #find locations where the value is 1.
        locations = np.where(thr_val >= 1)
        print(f'done thresholding! {len(locations[0])} potential storms')

        #close files
        thr_dvv.close()
        dvv_ds.close()

        #create pandas dataframe
        df = pd.DataFrame(columns=['x', 'y', 'Time', 'UH'])

        xs = []
        ys = []
        timess=[]
        refcs=[]
        uhs=[]
        uvvs=[]
        dvvs=[]

        #iterate through all potential center points. 
        for point in range(len(locations[0])):

            #save important attribute values for center points
            time = locations[0][point]
            y = locations[1][point] 
            x = locations[2][point]

            #append values to a list
            str_time = np.datetime_as_string(times[time])

            xs.append(x)
            ys.append(y)
            timess.append(str_time)
            refcs.append(refc_val[time,y,x])
            uvvs.append(uvv_val[time,y,x])
            dvvs.append(dvv_val[time,y,x])
            uhs.append(uh_val[time,y,x])

        #add list to pandas dataframe
        df['x'] = xs
        df['y'] = ys
        df['Time'] = timess
        df['DBZ'] = refcs
        df['UH'] = uhs
        df['UVV'] = uvvs
        df['DVV'] = dvvs

        #Save the dataframe as the csv.
        df.to_csv(f'/home/scratch/jcorner1/syn_sev/dataframes/HIST_DVV_{str_time[:4]}_{month}_threshold_dataframe.csv')



current year and month: 1990, 10
done thresholding! 1311 potential storms
current year and month: 1990, 11
done thresholding! 849 potential storms
current year and month: 1990, 12
done thresholding! 2054 potential storms
current year and month: 1990, 1
done thresholding! 25161 potential storms
current year and month: 1990, 2
done thresholding! 37507 potential storms
current year and month: 1990, 3
done thresholding! 32871 potential storms
current year and month: 1990, 4
done thresholding! 20349 potential storms
current year and month: 1990, 5
done thresholding! 53629 potential storms
current year and month: 1990, 6
done thresholding! 68388 potential storms
current year and month: 1990, 7
done thresholding! 100923 potential storms
current year and month: 1990, 8
done thresholding! 64029 potential storms
current year and month: 1990, 9
done thresholding! 11417 potential storms
current year and month: 1991, 10
done thresholding! 628 potential storms
current year and month: 1991, 11
done t